# Day 4: Agents and Tools


In [18]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

openai_client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=os.environ.get("GROQ_API_KEY")
)

user_prompt = "I just discovered the course, can I join now?"

chat_messages = [
    {"role": "user", "content": user_prompt}
]

response = openai_client.responses.create(
    model='llama-3.1-8b-instant',
    input=chat_messages,
)

print(response.output_text)


You're eager to start. Unfortunately, I'm a large language model, I don't have the ability to host or manage courses. This is a chat-based interface where we can discuss a wide range of topics, but I don't have specific courses or programs to enroll in.

However, if you're looking to learn something new or explore a particular topic, I'd be happy to help facilitate a conversation and provide information or resources to get you started! What subject or topic are you interested in learning more about?


In [19]:
import io
import zipfile
import requests
import frontmatter

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filename = file_info.filename
        filename_lower = filename.lower()

        if not (filename_lower.endswith('.md') 
            or filename_lower.endswith('.mdx')):
            continue
    
        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                post = frontmatter.loads(content)
                data = post.to_dict()
                data['filename'] = filename
                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue
    
    zf.close()
    return repository_data




from minsearch import Index

dtc_faq = read_repo_data('DataTalksClub', 'faq')

de_dtc_faq = [d for d in dtc_faq if 'data-engineering' in d['filename']]

faq_index = Index(
    text_fields=["question", "content"],
    keyword_fields=[]
)


def text_search(query):
    return faq_index.search(query, num_results=5)


In [20]:
text_search_tool = {
    "type": "function",
    "name": "text_search",
    "description": "Search the FAQ database",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Search query text to look up in the course FAQ."
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}


In [21]:
system_prompt = """
You are a helpful assistant for a course. 
"""

question = "I just discovered the course, can I join now?"

chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": question}
]

response = openai_client.responses.create(
    model='llama-3.1-8b-instant',
    input=chat_messages,
    tools=[text_search_tool]
)


In [22]:
import json

call = response.output[0]

arguments = json.loads(call.arguments)
result = text_search(**arguments)

call_output = {
    "type": "function_call_output",
    "call_id": call.call_id,
    "output": json.dumps(result),
}


AttributeError: 'ResponseReasoningItem' object has no attribute 'arguments'

In [ ]:
chat_messages.append(call)
chat_messages.append(call_output)

response = openai_client.responses.create(
    model='llama-3.1-8b-instant',
    input=chat_messages,
    tools=[text_search_tool]
)

print(response.output_text)


NameError: name 'call_output' is not defined

In [ ]:
from typing import List, Any

def text_search(query: str) -> List[Any]:
    """
    Perform a text-based search on the FAQ index.

    Args:
        query (str): The search query string.

    Returns:
        List[Any]: A list of up to 5 search results returned by the FAQ index.
    """
    return faq_index.search(query, num_results=5)


In [ ]:
from pydantic_ai import Agent

agent = Agent(
    name="faq_agent",
    instructions=system_prompt,
    tools=[text_search],
    model='groq:llama-3.3-70b-versatile'
)


In [ ]:
question = "I just discovered the course, can I join now?"

result = await agent.run(user_prompt=question)


NameError: name 'faq_index' is not defined

In [ ]:
import asyncio

result = asyncio.run(agent.run(user_prompt=question))


In [ ]:
result.new_messages()
